# M8. Ejercicio 3

https://pypi.org/project/PyMySQL/

Introduzco los parametros de conexión como indica la documentación de la libreria pero lo hago con un diccionario y luego los desempaqueto.
En Python, los dos puntos seguidos (**) se utilizan para desempaquetar un diccionario. En este contexto, **db_settings está desempaquetando el diccionario db_settings y pasando sus elementos como argumentos de palabra clave a la función pymysql.connect(). Esto es útil cuando tienes un diccionario con argumentos que quieres pasar a una función como argumentos de palabra clave. En este caso, pymysql.connect() espera argumentos de palabra clave como host, port, user, password, etc., y **db_settings hace que estos argumentos se pasen correctamente.

In [1]:
import pymysql
import pandas as pd

database_host = 'db.relational-data.org'
username = 'guest'
password = 'relational'
database_name = 'employee'
port = 3306

# Datos de conexión a la base de datos
db_settings = {
    "host": database_host, 
    "port": port,
    "user": username, 
    "password": password,
    "db": database_name, 
}

# Crear la conexión
connection = pymysql.connect(**db_settings)

In [2]:
query = '''
            SELECT 
                e.gender,
                t.title,
                ROUND(MAX(s.salary), 2) AS max_salary,
                ROUND(MIN(s.salary), 2) AS min_salary,
                ROUND(AVG(s.salary), 2) AS avg_salary
            FROM employees as e
            LEFT JOIN titles as t ON e.emp_no = t.emp_no
            LEFT JOIN salaries as s ON e.emp_no = s.emp_no
            GROUP BY 1, 2
        '''

# Ejecutar la consulta y obtener los resultados en un DataFrame de pandas
df = pd.read_sql(query, connection).sort_values(['title', 'gender'])

# Mostrar el resultado
display(df)

C:\Users\ferna\AppData\Local\Temp\ipykernel_7048\397836895.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection).sort_values(['title', 'gender'])


,gender,title,max_salary,min_salary,avg_salary
8,F,Assistant Engineer,117968,38850,58964.99
7,M,Assistant Engineer,133712,39089,59522.61
9,F,Engineer,138273,38849,59452.37
2,M,Engineer,140784,38851,59545.21
13,F,Manager,93193,40000,62037.22
12,M,Manager,108407,40000,72810.95
5,F,Senior Engineer,138273,38786,60467.82
0,M,Senior Engineer,140784,38851,60593.65
6,F,Senior Staff,152710,38923,70468.82
3,M,Senior Staff,158220,38735,70472.17


Con la ayuda de ChatGPT he usado plotly para pintar un grafico interactivo en el que podemos ver con los salarios para cada puesto (maximos, minimos y medios) suelen ser mayores para hombres sobre todo en el puesto de manager.

In [3]:
import plotly.graph_objects as go
import pandas as pd

# Suponiendo que 'df' contiene tus datos

# Crear una copia del DataFrame original para cada tipo de salario
df_avg = df.copy()
df_min = df.copy()
df_max = df.copy()

# Renombrar las columnas para cada tipo de salario
df_avg.rename(columns={'avg_salary': 'salary'}, inplace=True)
df_min.rename(columns={'min_salary': 'salary'}, inplace=True)
df_max.rename(columns={'max_salary': 'salary'}, inplace=True)

# Agregar una nueva columna para cada tipo de salario
df_avg['salary_type'] = 'Salario Promedio'
df_min['salary_type'] = 'Salario Mínimo'
df_max['salary_type'] = 'Salario Máximo'

# Concatenar los DataFrames
df_concat = pd.concat([df_avg, df_min, df_max])

# Agrupar los datos por puesto y tipo de salario para cada género
grouped_data = df_concat.groupby(['title', 'gender', 'salary_type']).agg({'salary': 'mean'}).reset_index()

# Crear el gráfico
fig = go.Figure()

# Definir colores para cada género
colors = {'M': 'blue', 'F': 'red'}

# Iterar sobre los géneros para agregar las barras correspondientes
for gender in grouped_data['gender'].unique():
    for salary_type in ['Salario Promedio', 'Salario Mínimo', 'Salario Máximo']:
        df_filtered = grouped_data[(grouped_data['gender'] == gender) & (grouped_data['salary_type'] == salary_type)]
        fig.add_trace(go.Bar(
            x=df_filtered['title'],
            y=df_filtered['salary'],
            name=f'{gender} - {salary_type}',
            marker_color=colors[gender],  # Seleccionar color según género
            legendgroup=gender,
            visible=False  # Inicialmente ocultamos todas las barras
        ))

# Mostrar las barras del primer género
fig.data[0].visible = True
fig.data[3].visible = True


# Crear botones para cambiar el tipo de salario
buttons = []
for i, salary_type in enumerate(['Salario Promedio', 'Salario Mínimo', 'Salario Máximo']):
    button = dict(label=salary_type,
                  method='update',
                  args=[{'visible': [i == j or (j >= 3 and j % 3 == i) for j in range(len(fig.data))]}])
    buttons.append(button)

# Actualizar el diseño del gráfico
fig.update_layout(
    barmode='group',
    title='Comparación del Salario por Género y Puesto',
    xaxis=dict(title='Puesto', tickangle=45),
    yaxis=dict(title='Salario'),
    updatemenus=[
        dict(buttons=buttons,
             direction='down',
             pad={'r': 10, 't': 10},
             showactive=True,
             x=0.1,
             xanchor='left',
             y=1.1,
             yanchor='top')
    ]
)

fig.show()

Por último, printeo un pequeño resumen de las diferencias entre salarios.

In [4]:
for i in ['max_salary', 'min_salary', 'avg_salary']:
    pivoted_df = df.pivot_table(index='title', columns='gender', values=[i]).reset_index()
    pivoted_df['diferencia'] = pivoted_df[(i, 'M')] - pivoted_df[(i, 'F')]
    if i == 'max_salary': pat ='salario máximo'
    elif i == 'min_salary': pat = 'salario mínimo'
    elif i == 'avg_salary': pat = 'salario medio'
    print(pat.upper())
    for j in range(pivoted_df.shape[0]):
        print(f'En el puesto {pivoted_df.iloc[j]['title'][0]} la diferencia de {pat} de un hombre frente a una mujer es de: {pivoted_df.iloc[j]['diferencia'][0]:.0f}€')



SALARIO MÁXIMO
En el puesto Assistant Engineer la diferencia de salario máximo de un hombre frente a una mujer es de: 15744€
En el puesto Engineer la diferencia de salario máximo de un hombre frente a una mujer es de: 2511€
En el puesto Manager la diferencia de salario máximo de un hombre frente a una mujer es de: 15214€
En el puesto Senior Engineer la diferencia de salario máximo de un hombre frente a una mujer es de: 2511€
En el puesto Senior Staff la diferencia de salario máximo de un hombre frente a una mujer es de: 5510€
En el puesto Staff la diferencia de salario máximo de un hombre frente a una mujer es de: 5533€
En el puesto Technique Leader la diferencia de salario máximo de un hombre frente a una mujer es de: -12201€
SALARIO MÍNIMO
En el puesto Assistant Engineer la diferencia de salario mínimo de un hombre frente a una mujer es de: 239€
En el puesto Engineer la diferencia de salario mínimo de un hombre frente a una mujer es de: 2€
En el puesto Manager la diferencia de salari

C:\Users\ferna\AppData\Local\Temp\ipykernel_7048\2299883907.py:9: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\ferna\AppData\Local\Temp\ipykernel_7048\2299883907.py:9: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\ferna\AppData\Local\Temp\ipykernel_7048\2299883907.py:9: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

